In [1]:
import numpy as np
import pandas as pd
import seaborn as sns

----


Pandas chose to use sentinels for missing data, and further chose to use two already-existing Python null values
* the special floating-point NaN value, 
* the Python None object. 

This choice has some side effects, as we will see.

### ``None``: Pythonic missing data

The first sentinel value used by Pandas is ``None``, a Python singleton object that is often used for missing data in Python code.

Because it is a Python object, ``None`` cannot be used in any arbitrary NumPy/Pandas array, but only in arrays with data type ``'object'`` (i.e., arrays of Python objects):

In [4]:
# dtype of the arr1 is "object"

arr1 = np.array([1, None, 3, 4])
arr1

array([1, None, 3, 4], dtype=object)

In [5]:
arr1.sum()

TypeError: unsupported operand type(s) for +: 'int' and 'NoneType'

The use of Python objects in an array also means that if you perform aggregations like ``sum()`` or ``min()`` across an array with a ``None`` value, you will generally get an error:

Other key impact of using dtype ``object`` is performance overhead. Whi

In [6]:
# 1E6 means 1000000. Short form
# Performance test

for dtype in ['object', 'int']:
    print("dtype =", dtype)
    %timeit np.arange(1E6, dtype=dtype).sum()
    print()

dtype = object
58.2 ms ± 1.38 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)

dtype = int
1.89 ms ± 177 µs per loop (mean ± std. dev. of 7 runs, 1000 loops each)



---

### ``NaN``: Missing numerical data

The other missing data representation, ``NaN`` (acronym for *Not a Number*), is different; it is a special floating-point value recognized by all systems that use the standard IEEE floating-point representation:

In [7]:
# Data type of arr2 os float 

arr2 = np.array([1, np.nan, 3, 4]) 
arr2.dtype

dtype('float64')

Regardless of the operation, the result of arithmetic with ``NaN`` will be another ``NaN``

In [8]:
print(arr2.sum(), arr2.min(), arr2.max())

nan nan nan


NumPy does provide some special aggregations that will ignore these missing values:

In [9]:
np.nansum(arr2), np.nanmin(arr2), np.nanmax(arr2)

(8.0, 1.0, 4.0)

### NaN and None in Pandas

``NaN`` and ``None`` both have their place, and Pandas is built to handle the two of them nearly interchangeably, converting between them where appropriate:

In [10]:
pd.Series([1, np.nan, 2, None])

0    1.0
1    NaN
2    2.0
3    NaN
dtype: float64

For types that don't have an available sentinel value, Pandas automatically type-casts when NA values are present.

For example, if we set a value in an integer array to ``np.nan``, it will automatically be upcast to a floating-point type to accommodate the NA. Also notice that in addition to casting the integer array to floating point, Pandas automatically converts the ``None`` to a ``NaN`` value.

In [11]:
x = pd.Series(range(2), dtype=int)
x

0    0
1    1
dtype: int32

In [12]:
x[0] = None
x

0    NaN
1    1.0
dtype: float64

---

## Operating on Null Values

Pandas treats ``None`` and ``NaN`` as essentially interchangeable for indicating missing or null values.

To facilitate this convention, there are several useful methods for detecting, removing, and replacing null values in Pandas data structures.
They are:

- ``isnull()``: Generate a boolean mask indicating missing values
- ``notnull()``: Opposite of ``isnull()``
- ``dropna()``: Return a filtered version of the data
- ``fillna()``: Return a copy of the data with missing values filled or imputed

### Detecting null values

Pandas data structures have two useful methods for detecting null data: ``isnull()`` and ``notnull()``.

Either one will return a Boolean mask over the data. For example:

In [13]:
data = pd.Series([1, np.nan, 'hello', None])

In [14]:
data.isnull()

0    False
1     True
2    False
3     True
dtype: bool

In [15]:
data.notnull()

0     True
1    False
2     True
3    False
dtype: bool

To display all `NaN` and `None` use `.isnull()`. To display the data without `NaN` and `None` use `.notnull()`.

In [16]:
# Display only NaN and None - To update with values

data[data.isnull()]

1     NaN
3    None
dtype: object

In [17]:
# To ignore the NaN and None and compute

data[data.notnull()]

0        1
2    hello
dtype: object

### Dropping null values

In addition to the masking used before, there are the convenience methods, ``dropna()`` (which removes NA values) and ``fillna()`` (which fills in NA values). For a ``Series``, the result is straightforward:

In [18]:
# Drops all the elements with is NaN and None
# It returns the delted array. Orginal not modified

print("Before deletion\n",data)
temp = data.dropna()
print("\nAfter Deletion\n",temp)

Before deletion
 0        1
1      NaN
2    hello
3     None
dtype: object

After Deletion
 0        1
2    hello
dtype: object


For a ``DataFrame``, there are more options as it has multiple axis. Consider the following ``DataFrame``:

In [19]:
df = pd.DataFrame([[1,      np.nan, 2],
                   [2,      3,      5],
                   [np.nan, 4,      6]])
df

,0,1,2
0,1.0,NaN,2
1,2.0,3.0,5
2,NaN,4.0,6


**We cannot drop single values** from a ``DataFrame``. we can only drop full rows or full columns. By default, ``dropna()`` will drop all rows in which *any* null value is present:

In [20]:
# Drop all the rows with NaN or None in any of its columns
# Original data not modified. 

df.dropna()

,0,1,2
1,2.0,3.0,5


In [21]:
# The above statement is equal to 

print(df.dropna(axis=0))
print("\n")
print(df.dropna(axis="rows"))

     0    1  2
1  2.0  3.0  5


     0    1  2
1  2.0  3.0  5


In [22]:
# Drop by columns


print(df.dropna(axis=1))
print("\n")
print(df.dropna(axis="columns"))

   2
0  2
1  5
2  6


   2
0  2
1  5
2  6


But this drops some good data as well; you might rather be interested in dropping rows or columns with *all* NA values, or a majority of NA values.
This can be specified through the ``how`` or ``thresh`` parameters, which allow fine control of the number of nulls to allow through.

The default is ``how='any'``, such that any row or column (depending on the ``axis`` keyword) containing a null value will be dropped.
You can also specify ``how='all'``, which will only drop rows/columns that are *all* null values:

In [23]:
# Add a column and row with all NaN
df[3]=np.nan
df[4]=np.nan
df.iloc[1,3]=19

#df.loc[len(df.index)] = [np.nan, np.nan, np.nan,np.nan]
df

,0,1,2,3,4
0,1.0,NaN,2,NaN,NaN
1,2.0,3.0,5,19.0,NaN
2,NaN,4.0,6,NaN,NaN


In [27]:
# Removes the column with all NaN's (4)

df.dropna(axis='columns', how='all')

,0,1,2,3
0,1.0,NaN,2,NaN
1,2.0,3.0,5,19.0
2,NaN,4.0,6,NaN


For finer-grained control, the ``thresh`` parameter lets you specify a minimum number of non-null values for the row/column to be kept:

In [29]:
# parameter thresh = Require that many non-NA values
# Thresh 1 means atleast one value is available in the column
# Hence column 1 and 3 is maintained

df.dropna(axis='columns', thresh=1)

,0,1,2,3
0,1.0,NaN,2,NaN
1,2.0,3.0,5,19.0
2,NaN,4.0,6,NaN


In [33]:
# parameter thresh = Require that many non-NA values
# Thresh 2 means atleast two value is available in the column
# Hence column 1 and 3 is maintained

df.dropna(axis='rows', thresh=3)

,0,1,2,3,4
1,2.0,3.0,5,19.0,NaN



---


### Filling null values

Sometimes rather than dropping NA values, you'd rather replace them with a valid value.
This value might be a single number like zero, or it might be some sort of imputation or interpolation from the good values.
You could do this in-place using the ``isnull()`` method as a mask, but because it is such a common operation Pandas provides the ``fillna()`` method, which returns a copy of the array with the null values replaced.

Consider the following ``Series``:

In [34]:
data = pd.Series([1, np.nan, 2, None, 3], index=list('abcde'))
data

a    1.0
b    NaN
c    2.0
d    NaN
e    3.0
dtype: float64

In [36]:
# Fill the NaN's with 0

data.fillna(0)

a    1.0
b    0.0
c    2.0
d    0.0
e    3.0
dtype: float64

In [38]:
# Propagate the previous value forward
# Uses the previous element value to fill

data.fillna(method='ffill')

a    1.0
b    1.0
c    2.0
d    2.0
e    3.0
dtype: float64

In [39]:
# Propagate the next values backward
# Uses the next element value to fill

data.fillna(method='bfill')

a    1.0
b    2.0
c    2.0
d    3.0
e    3.0
dtype: float64

In [44]:
df

,0,1,2,3,4
0,1.0,NaN,2,NaN,NaN
1,2.0,3.0,5,19.0,NaN
2,NaN,4.0,6,NaN,NaN


In [47]:
# Notice that if a previous value is not available 
# during a forward fill, the NA value remains.

df.fillna(method='ffill', axis=1)

,0,1,2,3,4
0,1.0,1.0,2.0,2.0,2.0
1,2.0,3.0,5.0,19.0,19.0
2,NaN,4.0,6.0,6.0,6.0



---
